<br></br>

<div align="center">

<h1 align="center">
    Cuaderno para el trabajo de clasificación relacional
    <br></br>
    Estudio de métricas de centralidad
</h1>

<h6 align="center">
    Antonio Macías Ferrera (antmacfer1@alum.us.es)
    <br></br>
    Delfín Santana Rubio (delsanrub@alum.us.es)
    <br></br>
    Universidad de Sevilla
</h6>

<br></br>

## **Control de Versiones**
    
| **Fecha**  | **Versión** | **Descripción**               |
| :--------- | :---------- | :---------------------------- |
| 27/05/2024 | v1r0        | Primera versión del cuaderno. |
| 29/05/2024 | v1r1        | Refactorización e inclsuión de otras medidas de centralidad.      |
| 07/06/2024 | v1r2        | Correcciones de formato.      |


</div>

<br></br>

## **Índice de contenido**

1. [Introducción](#introducción)
2. [Instanciación del grafo](#creacion)
3. [Implementación de funciones](#creacion)
4. [Estudio de las métricas](#estudio)
   - [Centralidad de grado](#degree)
   - [Centralidad k-path](#k-path)
   - [Centralidad de cercanía](#cercania)
   - [Centralidad de intermeciadión](intermeciadion#)
   - [Centralidad harmónica](#harmonica)

<br></br>

<br></br>

# <a name="introducción"></a> 1. **Introducción**

En este cuadernillo se encuentra todo el código utilizado para comprobar métricas de centralidad. Se han implementado dos funciones que calculan la métrica proporcinada para cada tipo de datos, y otra que evalúa la métrica haciendo uso del t-test Student.

<br></br>

# <a name="creacion"></a> 2. **Instanciación del grafo**

<br></br>

In [21]:
import matplotlib.pyplot as plt
import networkx as nx
import json
import csv

# Step 1: Read the edges.csv file
with open('../facebook_large/musae_facebook_edges.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    edges = [tuple(row) for row in edges_data]

# Step 3: Create a NetworkX graph object
G = nx.Graph()

# Step 4: Add nodes and edges to the graph
G.add_edges_from(edges)

tvshow = []
government = []
politician = []
company = []

# Step 5: Assign features to nodes or edges
with open('../facebook_large/musae_facebook_target.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    for row in edges_data:
        target = row[-1]
        node = row[0]
        if target == "government":
            government.append(node)
        elif target == "tvshow":
            tvshow.append(node)
        elif target == "politician":
            politician.append(node)
        elif target == "company":
            company.append(node)
            
        G.nodes[node]["target"] = target



# <a name="creacion"></a> 3. **Implementación de funciones**


En primer lugar, se ha tenido que calcular la densidad del grafo, ya que algunas medidas de centralidad adquieren una complejidad computacinal demasiado grande si el grafo es denso, así que en ese caso no sería rentable usar esa métrica.

In [22]:
# Determinar si el grafo es denso o disperso
density = nx.density(G)
umbral = 0.5
if density >= umbral:
    print(f"Densidad del grafo: {density}: El grafo es denso")
else:
    print(f"Densidad del grafo: {density}: El grafo es disperso")

Densidad del grafo: 0.000677398715568023: El grafo es disperso


Como se ha podido comprobar, el grafo es disperso, por lo que se podrán evaluar todas las medidas de centralidad: **centralidad de grado, cercanía, intermediación y harmónica**.

<br></br>

La función ```calc_metrics(results_dict)``` calcula la medida de centralidad dada para cada conjunto de datos del mismo tipo. Para cada nodo, se obtiene la métrica y se realiza la media y la desviación típica por tipo (). Así, con la desviación típica podemos ver si una medida es relativamente coherente en cada grupo. Si una métrica tiene una desviación típica pequeña en un grupo, quiere decir que esa métrica es similar para ese grupo, por lo que puede servir para diferenciarlo; mientras que si la desviación típica es grande, querrá decir que esa métrica es muy dispar dentro de ese grupo, por lo que no arrojará un resultado concluyente.

Por otro lado, la función ```t_Student_test(results_dict)``` genera un t-test Student que compara una métrica dada de cada grupo con el resto de grupos. Esto se hace para poder evaluar si una métrica es suficientemente diferente entre los distintos grupos para poder ser usada como atributo de entrenamiento del modelo. 

In [23]:
from numpy import std
from numpy import average
from scipy.stats import ttest_ind
from itertools import combinations

def calc_metrics(results_dict):
    tvshow_results = [results_dict[x] for x in tvshow]
    government_results = [results_dict[x] for x in government]
    politician_results = [results_dict[x] for x in politician]
    company_results = [results_dict[x] for x in company]

    tvshow_avg = average(tvshow_results)
    tvshow_deviation = std(tvshow_results)
    print("TVShow average= ",tvshow_avg," , deviation= ",tvshow_deviation)

    government_avg = average(government_results)
    government_deviation = std(government_results)
    print("Government average= ",government_avg," , deviation= ",government_deviation)

    politician_avg = average(politician_results)
    politician_deviation = std(politician_results)
    print("Politician average= ",politician_avg," , deviation= ",politician_deviation)

    company_avg = average(company_results)
    company_deviation = std(company_results)
    print("Company average= ",company_avg," , deviation= ",company_deviation)



# Calcular el test de t de Student
def t_Student_test(results_dict):
    tvshow_results = [results_dict[x] for x in tvshow]
    government_results = [results_dict[x] for x in government]
    politician_results = [results_dict[x] for x in politician]
    company_results = [results_dict[x] for x in company]
    datasets = [tvshow_results, government_results, politician_results, company_results]
    res = {}

    for (i, data_i), (j, data_j) in combinations(enumerate(datasets), 2):
        t_stat, p_value = ttest_ind(data_i, data_j)
        res[f"data{i+1} vs data{j+1}"] = (t_stat, p_value)

    for key, value in res.items():
        print(f"{key}: T-statistic = {value[0]}, P-value = {value[1]}")


In [24]:
from itertools import permutations

# Calcular k-path centrality
def k_path_centrality(G, k):
    # Inicializar el diccionario para almacenar la centralidad
    centrality = {node: 0 for node in G.nodes()}
    
    # Generar todos los k-paths posibles en el grafo
    for path in permutations(G.nodes(), k + 1):
        # Verificar si es un camino válido (todos los nodos consecutivos están conectados)
        valid_path = all(G.has_edge(path[i], path[i+1]) for i in range(k))
        if valid_path:
            # Incrementar el contador para cada nodo en el camino
            for node in path:
                centrality[node] += 1
                
    # Normalizar la centralidad dividiendo por el número total de caminos generados
    total_paths = sum(centrality.values())
    if total_paths > 0:
        for node in centrality:
            centrality[node] /= total_paths
            
    return centrality

# <a name="estudio"></a> 4. **Estudio de las métricas**

En este apartado se evaluará cada métrica para hacer una selección 'a priori'. Se pretende descartar métricas que aporten muy mal resultado en el Student test.

### <a name="degree"></a> **Centralidad de grado**


In [25]:
# Centralidad de grado 
degree_centrality = nx.degree_centrality(G)
calc_metrics(degree_centrality)
t_Student_test(degree_centrality)

TVShow average=  0.0004139156399473299  , deviation=  0.0005789308691489977
Government average=  0.0011507715539860086  , deviation=  0.0017220288003390594
Politician average=  0.0006559276090608796  , deviation=  0.0009597952783473788
Company average=  0.00033000047178165  , deviation=  0.0005060151975631529
data1 vs data2: T-statistic = -24.030932285131954, P-value = 3.5694383416148093e-124
data1 vs data3: T-statistic = -13.22116833880333, P-value = 1.5324464067910483e-39
data1 vs data4: T-statistic = 7.400066389711722, P-value = 1.472791219967126e-13
data2 vs data3: T-statistic = 19.4376821783884, P-value = 5.983198798280145e-83
data2 vs data4: T-statistic = 36.93367188542069, P-value = 1.9868984900564964e-284
data3 vs data4: T-statistic = 23.881750500712428, P-value = 3.0185755342125682e-123


### <a name="k-path"></a> **Centralidad k-path**


In [26]:
# Centralidad k-path (k=1)
k_path_centrality = k_path_centrality(G, k=1)
t_Student_test(k_path_centrality)

data1 vs data2: T-statistic = -24.040910629465056, P-value = 2.8434791373913746e-124
data1 vs data3: T-statistic = -13.239732634092194, P-value = 1.2026521731460227e-39
data1 vs data4: T-statistic = 7.482597902453403, P-value = 7.912022708112524e-14
data2 vs data3: T-statistic = 19.435820337441378, P-value = 6.197749635803934e-83
data2 vs data4: T-statistic = 36.990057395792384, P-value = 2.9946931031275e-285
data3 vs data4: T-statistic = 23.97654606702883, P-value = 3.4433049199437596e-124


### <a name="cercania"></a> **Centralidad de cercanía**


In [27]:
# Centralidad de cercanía
closeness_centrality = nx.closeness_centrality(G)
t_Student_test(closeness_centrality)

data1 vs data2: T-statistic = -60.655831548304114, P-value = 0.0
data1 vs data3: T-statistic = -24.979331210097637, P-value = 2.9638672148870546e-133
data1 vs data4: T-statistic = -5.6098977464643225, P-value = 2.0793267902669518e-08
data2 vs data3: T-statistic = 46.255904429060436, P-value = 0.0
data2 vs data4: T-statistic = 63.64625690544811, P-value = 0.0
data3 vs data4: T-statistic = 21.009086396452876, P-value = 2.668331706585062e-96


### <a name="intermediacion"></a> **Centralidad de intermediación**


In [ ]:
# Centralidad de intermeciadión (APORTA UN MAL RESULTADO, POR ENCIAMA DE 0.05)
betweenness_centrality = nx.betweenness_centrality(G)
t_Student_test(betweenness_centrality)

### <a name="harmonica"></a> **Centralidad harmónica**


In [ ]:
# Centralidad harmonica
harmonic_centrality = nx.harmonic_centrality(G)
calc_metrics(harmonic_centrality)
t_Student_test(harmonic_centrality)

Tras estudiar las métricas anteriores observamos que la única que aporta un mal resultado de p-value en el Student test es la 'betweeness centrality', con lo cual sería la métrica de centralidad que se podría omitir 'a priori'.